In [ ]:
import sys

%load_ext autoreload
%load_ext line_profiler
%autoreload 2


sys.path.append("..")

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from joblib import Parallel, delayed
from matplotlib import rc
from problems import FDS, FDS_CONSTRAINED, JOS1, JOS1_L1, SD
from tqdm.notebook import tqdm
from zfista import minimize_proximal_gradient

In [ ]:
def generate_start_points(low, high, n_dims, n_samples=1000):
    return [
        np.random.uniform(low=low, high=high, size=n_dims) for _ in range(n_samples)
    ]


def run(
    problem,
    start_points,
    tol=1e-5,
    nesterov=False,
    nesterov_ratio=(0, 0.25),
    n_jobs=-1,
    verbose=False,
):
    results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(minimize_proximal_gradient)(
            problem.f,
            problem.g,
            problem.jac_f,
            problem.prox_wsum_g,
            x0,
            tol=tol,
            nesterov=nesterov,
            nesterov_ratio=nesterov_ratio,
            return_all=True,
            verbose=verbose,
        )
        for x0 in start_points
    )
    return results


def show_Pareto_front(
    problem,
    results,
    results_nesterov,
    step=None,
    s=15,
    alpha=0.75,
    fname=None,
    elev=15,
    azim=130,
    linewidths=0.1,
):
    labels = [
        "Starting points",
        f"PGM ($k={step}$)",
        f"Acc-PGM ($k={step}$)",
        "PGM (Solutions)",
        "Acc-PGM (Solutions)",
    ]
    fig = plt.figure(figsize=(7.5, 7.5), dpi=100)
    if problem.m_dims == 2:
        ax = fig.add_subplot(111)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    if problem.m_dims == 3:
        ax = fig.add_subplot(111, projection="3d", clip_on=True)
        ax.view_init(elev=elev, azim=azim)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=0.6)
    for i, (result, result_acc) in tqdm(enumerate(zip(results, results_nesterov))):
        allvecs = result.allvecs
        allvecs_acc = result_acc.allvecs
        x0 = allvecs[0]
        F_of_x0 = problem.f(x0) + problem.g(x0)
        if step is not None:
            xk = allvecs[step]
            xk_acc = allvecs_acc[step]
            F_of_xk = problem.f(xk) + problem.g(xk)
            F_of_xk_acc = problem.f(xk_acc) + problem.g(xk_acc)
        F_pareto = result.fun
        F_pareto_acc = result_acc.fun
        if problem.m_dims == 2:
            ax.scatter(
                *F_of_x0,
                color="#8e44ad",
                marker="x",
                label=labels[0],
                s=s,
                alpha=alpha,
                linewidths=linewidths,
            )
            if step is not None:
                ax.scatter(
                    *F_of_xk,
                    color="#2980b9",
                    marker="<",
                    label=labels[1],
                    s=s,
                    alpha=alpha,
                    linewidths=linewidths,
                )
                ax.scatter(
                    *F_of_xk_acc,
                    facecolors="none",
                    edgecolor="#e74c3c",
                    marker="*",
                    label=labels[2],
                    s=s,
                    alpha=alpha,
                    linewidths=linewidths,
                )
        ax.scatter(
            *F_pareto,
            color="#2980b9",
            marker=".",
            label=labels[3],
            s=s,
            alpha=alpha,
            linewidths=linewidths,
        )
        ax.scatter(
            *F_pareto_acc,
            facecolors="none",
            edgecolors="#e74c3c",
            marker="D",
            label=labels[4],
            s=s,
            alpha=alpha,
            linewidths=linewidths,
        )
    ax.set_xlabel(r"$F_1$", fontsize=15)
    ax.set_ylabel(r"$F_2$", fontsize=15)
    if problem.m_dims == 3:
        ax.set_zlabel(r"$F_3$", fontsize=15)
        ax.legend(labels[-2:], bbox_transform=ax.transData)
    elif step is None:
        ax.legend([labels[0]] + labels[-2:])
    else:
        ax.legend(labels)
    if fname is not None:
        plt.savefig(fig_path + "/" + fname, bbox_inches="tight")


def get_stats(results):
    nits = [result.nit for result in results]
    nit_internals = [result.nit_internal for result in results]
    execution_times = [result.execution_time for result in results]
    stats = {
        "nit": {"mean": np.mean(nits), "std": np.std(nits), "max": np.max(nits)},
        "nit_internal": {
            "mean": np.mean(nit_internals),
            "std": np.std(nit_internals),
            "max": np.max(nit_internals),
        },
        "execusion_time": {
            "mean": np.mean(execution_times),
            "std": np.std(execution_times),
            "max": np.max(execution_times),
        },
    }
    return stats

## JOS1
Minimize
$$
f_1(x) = \frac{1}{n} \| x \|_2^2, \quad f_2(x) = \frac{1}{n} \| x - 2\|_2^2
$$
subject to $x \in \mathbf{R^n}$.

In [ ]:
n_dims = 50
problem_JOS1 = JOS1(n_dims=n_dims)
start_points_JOS1 = generate_start_points(low=-2, high=4, n_dims=n_dims)

### Proximal Gradient Method

In [ ]:
results_JOS1 = run(problem_JOS1, start_points_JOS1)

### Accelerated Proximal Gradient Method

In [ ]:
results_acc_JOS1 = run(problem_JOS1, start_points_JOS1, nesterov=True)

### Complexity

In [ ]:
import pprint

stats_JOS1 = {"PGM": get_stats(results_JOS1), "Acc-PGM": get_stats(results_acc_JOS1)}
pprint.pprint(stats_JOS1)

In [ ]:
show_Pareto_front(
    problem_JOS1, results_JOS1, results_acc_JOS1, step=10, fname="JOS1.pdf"
)

In [ ]:
plt.yscale("log")
plt.xlabel(r"$k$", fontsize=15)
plt.ylabel(r"$\|x^k - y^k\|_\infty$", fontsize=15)
plt.plot(
    results_JOS1[0].all_error_criteria, color="#2980b9", label="PGM", linestyle="dashed"
)
plt.plot(results_acc_JOS1[0].all_error_criteria, color="#e74c3c", label="Acc-PGM")
plt.legend()
plt.savefig(fig_path + "/JOS1_error.pdf", bbox_inches="tight")

## JOS1 + $\ell_1$ penalty
Minimize
$$
F_1(x) = \frac{1}{n} \| x \|_2^2 + \frac{1}{n} \|x\|_1, \quad F_2(x) = \frac{1}{n} \| x - 2\|_2^2 + \frac{1}{2n} \|x - 1\|_1
$$
subject to $x \in \mathbf{R}^n$.

In [ ]:
n_dims = 50
problem_JOS1_L1 = JOS1_L1(n_dims=n_dims, l1_ratios=(1 / n_dims, 1 / n_dims / 2))
start_points_JOS1_L1 = generate_start_points(low=-2, high=4, n_dims=n_dims)

### Proximal Gradient Method

In [ ]:
results_JOS1_L1 = run(problem_JOS1_L1, start_points_JOS1_L1)

### Accelerated Proximal Gradient Method

In [ ]:
results_acc_JOS1_L1 = run(problem_JOS1_L1, start_points_JOS1_L1, nesterov=True)

### Complexity

In [ ]:
stats_JOS1_L1 = {
    "PGM": get_stats(results_JOS1_L1),
    "Acc-PGM": get_stats(results_acc_JOS1_L1),
}
pprint.pprint(stats_JOS1_L1)

In [ ]:
show_Pareto_front(
    problem_JOS1_L1, results_JOS1_L1, results_acc_JOS1_L1, step=10, fname="JOS1_L1.pdf"
)

In [ ]:
plt.yscale("log")
plt.xlabel(r"$k$", fontsize=15)
plt.ylabel(r"$\|x^k - y^k\|_\infty$", fontsize=15)
plt.plot(
    results_JOS1_L1[0].all_error_criteria,
    color="#2980b9",
    label="PGM",
    linestyle="dashed",
)
plt.plot(results_acc_JOS1_L1[0].all_error_criteria, color="#e74c3c", label="Acc-PGM")
plt.legend()
plt.savefig(fig_path + "/JOS1_L1_error.pdf", bbox_inches="tight")

## SD
Minimize
$$F_1(x) = 2 x_1 + \sqrt{2} x_2 + \sqrt{2} x_3 + x_4, \quad F_2(x) = \frac{2}{x_1} + \frac{2 \sqrt{2}}{x_2} + \frac{2 \sqrt{2}}{x_3} + \frac{2}{x_4}$$
subject to $(1, \sqrt{2}, \sqrt{2}, 1)^\top \le x \le (3, 3, 3, 3)^\top$.

In [ ]:
problem_SD = SD()
start_points_SD = generate_start_points(
    low=problem_SD.lb, high=problem_SD.ub, n_dims=problem_SD.n_dims
)

### Proximal Gradient Method

In [ ]:
results_SD = run(problem_SD, start_points_SD)

### Accelerated Proximal Gradient Method

In [ ]:
results_acc_SD = run(problem_SD, start_points_SD, nesterov=True)

### Complexity

In [ ]:
stats_SD = {"PGM": get_stats(results_SD), "Acc-PGM": get_stats(results_acc_SD)}
pprint.pprint(stats_SD)

In [ ]:
%matplotlib inline
show_Pareto_front(problem_SD, results_SD, results_acc_SD, fname="SD.pdf")

In [ ]:
plt.yscale("log")
plt.xlabel(r"$k$", fontsize=15)
plt.ylabel(r"$\|x^k - y^k\|_\infty$", fontsize=15)
plt.plot(
    results_SD[0].all_error_criteria, color="#2980b9", label="PGM", linestyle="dashed"
)
plt.plot(results_acc_SD[0].all_error_criteria, color="#e74c3c", label="Acc-PGM")
plt.legend()
plt.savefig(fig_path + "/SD_error.pdf", bbox_inches="tight")

## FDS
Minimize
$$F_1(x) = \frac{1}{n^2} \sum_{i = 1}^n i (x_i - i)^4, \quad F_2(x) = \exp \left( \sum_{i = 1}^n \frac{x_i}{n} \right) + \|x\|_2^2, \quad F_3(x) = \frac{1}{n(n + 1)} \sum_{i = 1}^n i (n - i + 1) \exp (- x_i)$$
subject to $x \in \mathbf{R}^n$.

In [ ]:
n_dims = 10
problem_FDS = FDS(n_dims=n_dims)
start_points_FDS = generate_start_points(low=-2, high=2, n_dims=n_dims)

### Proximal Gradient Method

In [ ]:
results_FDS = run(problem_FDS, start_points_FDS)

### Accelerated Proximal Gradient Method

In [ ]:
results_acc_FDS = run(problem_FDS, start_points_FDS, nesterov=True)

### Complexity

In [ ]:
stats_FDS = {"PGM": get_stats(results_FDS), "Acc-PGM": get_stats(results_acc_FDS)}
pprint.pprint(stats_FDS)

In [ ]:
%matplotlib inline
show_Pareto_front(problem_FDS, results_FDS, results_acc_FDS, fname="FDS.pdf")

In [ ]:
plt.yscale("log")
plt.xlabel(r"$k$", fontsize=15)
plt.ylabel(r"$\|x^k - y^k\|_\infty$", fontsize=15)
plt.plot(
    results_FDS[0].all_error_criteria, color="#2980b9", label="PGM", linestyle="dashed"
)
plt.plot(results_acc_FDS[0].all_error_criteria, color="#e74c3c", label="Acc-PGM")
plt.legend()
plt.savefig(fig_path + "/FDS_error.pdf", bbox_inches="tight")

## FDS CONSTRAINED
Minimize
$$F_1(x) = \frac{1}{n^2} \sum_{i = 1}^n i (x_i - i)^4, \quad F_2(x) = \exp \left( \sum_{i = 1}^n \frac{x_i}{n} \right) + \|x\|_2^2, \quad F_3(x) = \frac{1}{n(n + 1)} \sum_{i = 1}^n i (n - i + 1) \exp (- x_i)$$
subject to $x \in \mathbf{R}_+^n$.

In [ ]:
n_dims = 10
problem_FDS_CONSTRAINED = FDS_CONSTRAINED(n_dims=n_dims)
start_points_FDS_CONSTRAINED = generate_start_points(low=0, high=2, n_dims=n_dims)

In [ ]:
results_FDS_CONSTRAINED = run(problem_FDS_CONSTRAINED, start_points_FDS_CONSTRAINED)

In [ ]:
results_acc_FDS_CONSTRAINED = run(
    problem_FDS_CONSTRAINED, start_points_FDS_CONSTRAINED, nesterov=True
)

In [ ]:
stats_FDS_CONSTRAINED = {
    "PGM": get_stats(results_FDS_CONSTRAINED),
    "Acc-PGM": get_stats(results_acc_FDS_CONSTRAINED),
}
pprint.pprint(stats_FDS_CONSTRAINED)

In [ ]:
%matplotlib inline
show_Pareto_front(
    problem_FDS_CONSTRAINED,
    results_FDS_CONSTRAINED,
    results_acc_FDS_CONSTRAINED,
    fname="FDS_CONSTRAINED.pdf",
)

In [ ]:
plt.yscale("log")
plt.xlabel(r"$k$", fontsize=15)
plt.ylabel(r"$\|x^k - y^k\|_\infty$", fontsize=15)
plt.plot(
    results_FDS_CONSTRAINED[0].all_error_criteria,
    color="#2980b9",
    label="PGM",
    linestyle="dashed",
)
plt.plot(
    results_acc_FDS_CONSTRAINED[0].all_error_criteria, color="#e74c3c", label="Acc-PGM"
)
plt.legend()
plt.savefig(fig_path + "/FDS_CONSTRAINED_error.pdf", bbox_inches="tight")